# Interactive Logon to Server Systems
Detects interactive console logons to

## Rule Content
```
- title: Interactive Logon to Server Systems
  id: 3ff152b2-1388-4984-9cd9-a323323fdadf
  description: Detects interactive console logons to
  author: Florian Roth
  tags:
  - attack.lateral_movement
  - attack.t1078
  logsource:
    product: windows
    service: security
    category: null
  detection:
    selection:
      EventID:
      - 528
      - 529
      - 4624
      - 4625
      LogonType: 2
      ComputerName:
      - '%ServerSystems%'
      - '%DomainControllers%'
    filter:
      LogonProcessName: Advapi
      ComputerName: '%Workstations%'
    condition: selection and not filter
  falsepositives:
  - Administrative activity via KVM or ILO board
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-security-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((event_id:("528" OR "529" OR "4624" OR "4625") AND logon_type:"2" AND host_name:("%ServerSystems%" OR "%DomainControllers%")) AND (NOT (logon_process_name:"Advapi" AND host_name:"%Workstations%")))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()